In [2]:
# Import necessary libraries and custom PPO implementation
import numpy as np
import torch
import torch.nn as nn
import gymnasium as gym
import qwop_gym

# Reload train_ppo module to pick up latest changes
import importlib
import train_ppo
importlib.reload(train_ppo)

# Import custom PPO components from train_ppo.py
from train_ppo import (
    ActorCritic,
    RolloutBuffer,
    create_qwop_env,
    train_ppo,
    evaluate_policy,
    save_checkpoint,
    load_checkpoint
)

print("✓ All modules imported successfully!")
print(f"Using PyTorch device: {torch.device('cuda' if torch.cuda.is_available() else 'cpu')}")

✓ All modules imported successfully!
Using PyTorch device: cpu


In [3]:
# Custom Reward Wrapper
class CustomRewardWrapper(gym.Wrapper):
    """
    Wrapper to customize the reward function for QWOP.
    
    Default reward is typically based on distance traveled.
    You can modify this to add penalties, bonuses, or completely change the reward structure.
    """
    
    def __init__(self, env, penalty_scale=0.1, y_threshold=1.5, verbose=False):
        super().__init__(env)
        self.prev_distance = 0
        self.steps_taken = 0
        self.torso_y_idx = 1  # Index for torso Y-level in observation
        self.y_threshold = y_threshold  # Minimum acceptable torso height
        self.penalty_scale = penalty_scale  # Scale for posture penalty
        self.verbose = verbose  # Print reward breakdown occasionally
        self.total_base_reward = 0
        self.total_penalty = 0
        self.episode_steps = 0
    
    def reset(self, **kwargs):
        if self.verbose and self.episode_steps > 0:
            print(f"\n[CustomReward] Episode Summary:")
            print(f"  Total base reward: {self.total_base_reward:.2f}")
            print(f"  Total penalty: {self.total_penalty:.2f}")
            print(f"  Penalty scale: {self.penalty_scale}")
            print(f"  Average penalty per step: {self.total_penalty/self.episode_steps:.4f}")
        
        self.prev_distance = 0
        self.steps_taken = 0
        self.total_base_reward = 0
        self.total_penalty = 0
        self.episode_steps = 0
        return self.env.reset(**kwargs)
    
    def step(self, action):
        obs, base_reward, terminated, truncated, info = self.env.step(action)
        self.steps_taken += 1
        
       # Get Torso Y-Level (Need to ensure this index is correct)
        y_torso = obs[self.torso_y_idx]

        posture_penalty = 0
        if y_torso > self.y_threshold:
            # Linear Penalty: -k * (y_threshold - y_torso)
            posture_penalty = -self.penalty_scale * (self.y_threshold - y_torso)

        custom_reward = base_reward + posture_penalty
        
        # Track cumulative rewards for debugging
        self.total_base_reward += base_reward
        self.total_penalty += posture_penalty
        
        # Verbose logging every 100 steps
        if self.verbose and self.steps_taken % 100 == 0:
            print(f"Step {self.steps_taken}: y_torso={y_torso:.3f}, "
                  f"base_reward={base_reward:.3f}, penalty={posture_penalty:.3f}, "
                  f"total_reward={custom_reward:.3f}")
        
        return obs, custom_reward, terminated, truncated, info

print("✓ CustomRewardWrapper defined!")
print("\nKey features:")
print("  - Penalizes low torso height (bad posture)")
print("  - Adjustable penalty_scale parameter")
print("  - Set verbose=True to see reward breakdown")
print("\nUsage:")
print("  env = CustomRewardWrapper(env, penalty_scale=0.5, y_threshold=1.5, verbose=True)")


✓ CustomRewardWrapper defined!

Key features:
  - Penalizes low torso height (bad posture)
  - Adjustable penalty_scale parameter
  - Set verbose=True to see reward breakdown

Usage:
  env = CustomRewardWrapper(env, penalty_scale=0.5, y_threshold=1.5, verbose=True)


In [4]:
# Configuration for QWOP environment
BROWSER_PATH = "C:\\Program Files\\BraveSoftware\\Brave-Browser\\Application\\brave.exe"
DRIVER_PATH = "C:\\Program Files\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe"

# Create QWOP environment using the helper function
env = create_qwop_env(
    browser_path=BROWSER_PATH,
    driver_path=DRIVER_PATH,
    stat_in_browser=True,
    game_in_browser=True,
    auto_draw=True,
    frames_per_step=4,
    max_episode_steps=2000,
    text_in_browser="🤖 Training PPO Agent"
)

print(f"✓ Environment created successfully!")
print(f"State dimension: {env.observation_space.shape[0]}")
print(f"Action dimension: {env.action_space.n}")
print("Check your Brave browser - the QWOP game should be visible!")

✓ Environment created successfully!
State dimension: 60
Action dimension: 16
Check your Brave browser - the QWOP game should be visible!


In [5]:
# Action space mapping
action_map = {
    0: "none", 1: "Q", 2: "W", 3: "O", 4: "P",
    5: "Q+W", 6: "Q+O", 7: "Q+P", 8: "W+O", 9: "W+P", 10: "O+P",
    11: "Q+W+O", 12: "Q+W+P", 13: "Q+O+P", 14: "W+O+P", 15: "Q+W+O+P"
}

print(f"Testing environment with {env.action_space.n} actions\n")

obs, info = env.reset()
print(f"Initial observation shape: {obs.shape}")
print(f"Initial info: {info}\n")

# Take a few random actions
total_reward = 0
for step in range(50):
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    total_reward += reward
    
    if step % 10 == 0:
        print(f"Step {step}: Action={action_map.get(action, 'UNKNOWN')}, "
              f"Reward={reward:.2f}, Distance={info['distance']:.2f}m")
    
    if terminated or truncated:
        print(f"\nEpisode finished at step {step + 1}")
        print(f"Final distance: {info['distance']:.2f}m, Total reward: {total_reward:.2f}")
        break

print("\n✓ Environment test complete!")

Testing environment with 16 actions

Initial observation shape: (60,)
Initial info: {'time': np.float32(0.008433334), 'distance': np.float32(0.25110978), 'avgspeed': np.float32(29.775862), 'is_success': False}

Step 0: Action=Q+W+O+P, Reward=0.06, Distance=0.26m
Step 10: Action=Q+W+O+P, Reward=-0.03, Distance=0.39m

Episode finished at step 13
Final distance: 0.40m, Total reward: -10.21

✓ Environment test complete!


## Train PPO Agent

In [6]:
# Training configuration
import os
from pathlib import Path

# PPO Hyperparameters
TOTAL_STEPS = 25000
ROLLOUT_STEPS = 2048
BATCH_SIZE = 256
PPO_EPOCHS = 4
GAMMA = 0.995
GAE_LAMBDA = 0.95
CLIP_COEF = 0.2
LEARNING_RATE = 3e-4
VALUE_COEF = 0.5
ENTROPY_COEF = 0.01
MAX_GRAD_NORM = 0.5
HIDDEN_SIZE = 256

# Custom Reward Parameters
PENALTY_SCALE = 0.5  # Increase this to make penalty more significant (try 0.5, 1.0, 2.0)
Y_THRESHOLD = 1.5    # Torso height threshold (lower = more strict)
REWARD_VERBOSE = True  # Set to True to see reward breakdown

# Output directory
OUTPUT_DIR = "../data/PPO-notebook"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Create fresh environment for training
env = create_qwop_env(
    browser_path=BROWSER_PATH,
    driver_path=DRIVER_PATH,
    stat_in_browser=True,
    game_in_browser=True,
    auto_draw=True,
    frames_per_step=8,
    max_episode_steps=500,
    text_in_browser="🤖 Training PPO Agent"
)

# Apply custom reward wrapper with specified parameters
env = CustomRewardWrapper(env, penalty_scale=PENALTY_SCALE, y_threshold=Y_THRESHOLD, verbose=REWARD_VERBOSE)
print(f"✓ Custom reward wrapper applied!")
print(f"  Penalty scale: {PENALTY_SCALE}")
print(f"  Y threshold: {Y_THRESHOLD}")
print(f"  Verbose: {REWARD_VERBOSE}")

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
print(f"State dimension: {state_dim}, Action dimension: {action_dim}")

# Initialize policy and optimizer
policy = ActorCritic(state_dim, action_dim, HIDDEN_SIZE).to(device)
optimizer = torch.optim.Adam(policy.parameters(), lr=LEARNING_RATE, eps=1e-5)

print("\nStarting PPO training...")
print(f"Total steps: {TOTAL_STEPS:,}")
print(f"Rollout steps: {ROLLOUT_STEPS:,}")
print(f"Watch the browser to see the agent learning!\n")

# Train the agent using the train_ppo function
results = train_ppo(
    env=env,
    policy=policy,
    optimizer=optimizer,
    device=device,
    total_steps=TOTAL_STEPS,
    rollout_steps=ROLLOUT_STEPS,
    batch_size=BATCH_SIZE,
    ppo_epochs=PPO_EPOCHS,
    gamma=GAMMA,
    gae_lambda=GAE_LAMBDA,
    clip_coef=CLIP_COEF,
    value_coef=VALUE_COEF,
    entropy_coef=ENTROPY_COEF,
    max_grad_norm=MAX_GRAD_NORM,
    model_dir=OUTPUT_DIR,
    save_every=10000,
    verbose=True
)

# Print training summary
print("\n" + "="*60)
print("TRAINING SUMMARY")
print("="*60)
print(f"Total steps: {results['total_steps']:,}")
print(f"Episodes completed: {results['completed_episodes']}")
print(f"Average episode reward: {results['average_reward']:.2f}")
print(f"Average episode length: {results['average_length']:.1f}")
print(f"Final model saved to: {results['final_checkpoint']}")
print("="*60)

# Close environment
env.close()
print("\n✓ Training complete!")


Using device: cpu


✓ Custom reward wrapper applied!
  Penalty scale: 0.5
  Y threshold: 1.5
  Verbose: True
State dimension: 60, Action dimension: 16

Starting PPO training...
Total steps: 25,000
Rollout steps: 2,048
Watch the browser to see the agent learning!

episode 1: reward=-10.12 len=  14
episode 2: reward=-10.47 len=  18
episode 3: reward=-11.56 len=  21
episode 4: reward=-10.85 len=  33
episode 5: reward=-10.46 len=  12
episode 6: reward=-10.14 len=  11
episode 7: reward=-10.43 len=  15


-- 30199.66s [wsclient] WARNING: Failed to send/receive: timed out in 3.0s
-- 30199.66s [wsclient] WARNING: Failed to send/receive: timed out in 3.0s



[info] KeyboardInterrupt received; saving final checkpoint...
Training interrupted. Latest policy saved to ..\data\PPO-notebook\ppo_final.pt

TRAINING SUMMARY
Total steps: 132
Episodes completed: 7
Average episode reward: -10.58
Average episode length: 17.7
Final model saved to: ..\data\PPO-notebook\ppo_final.pt

✓ Training complete!


## Evaluate Trained Agent

Load the trained model and watch it play QWOP:

In [ ]:
# Load and evaluate the trained PPO agent
MODEL_PATH = "../data/PPO-notebook/custom_ppo_final.pt"  # Use custom trained model
HIDDEN_SIZE = 256

print(f"Loading model from: {MODEL_PATH}")

# Create evaluation environment
eval_env = create_qwop_env(
    browser_path=BROWSER_PATH,
    driver_path=DRIVER_PATH,
    stat_in_browser=True,
    game_in_browser=True,
    auto_draw=True,
    frames_per_step=4,
    max_episode_steps=5000,
    text_in_browser="🤖 PPO Agent Evaluation"
)

# Apply same custom reward wrapper (though rewards don't matter much during eval)
eval_env = CustomRewardWrapper(eval_env, penalty_scale=PENALTY_SCALE, y_threshold=Y_THRESHOLD, verbose=False)

# Initialize policy and load checkpoint
state_dim = eval_env.observation_space.shape[0]
action_dim = eval_env.action_space.n
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

eval_policy = ActorCritic(state_dim, action_dim, HIDDEN_SIZE).to(device)
checkpoint = load_checkpoint(MODEL_PATH, eval_policy, device)

print(f"✓ Model loaded (trained for {checkpoint['step']} steps)")
print("\nEvaluating agent for 5 episodes...")
print("Watch the browser to see your trained AI in action!\n")

# Evaluate the policy
results = evaluate_policy(
    env=eval_env,
    policy=eval_policy,
    device=device,
    num_episodes=5,
    verbose=True,
    render_delay=0.02
)

# Close environment
eval_env.close()
print("\n✓ Evaluation complete!")


Loading model from: ../data/PPO-notebook/custom_ppo_final.pt


NameError: name 'create_qwop_env' is not defined